# data analysis and wrangling


## Imports

In [58]:
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# preprocessing
from sklearn import preprocessing

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score


## Load Data

In [59]:
train_df = pd.read_csv('../dataset/train.csv')
test_df = pd.read_csv('../dataset/test.csv')
result_df = pd.read_csv('../dataset/gender_submission.csv')
combine = [train_df , test_df]
print(train_df.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


In [60]:
# for dataset in combine:
#     dataset['Mu'] = 1
#     dataset.loc[(dataset.Ticket.duplicated(keep= False)) , 'Mu'] = 0
#     dataset['Mu'] = dataset['Mu'].astype(int)
# train_df[['Mu', 'Survived']].groupby(['Mu'], as_index=False).mean()

In [61]:
print("Before", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

train_df = train_df.drop(['Cabin', 'Ticket'], axis=1)
test_df = test_df.drop([ 'Cabin', 'Ticket'], axis=1)
combine = [train_df, test_df]

"After", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape

Before (891, 12) (418, 11) (891, 12) (418, 11)


('After', (891, 10), (418, 9), (891, 10), (418, 9))

In [62]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    
pd.crosstab(train_df['Title'], train_df['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [63]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Mlle', 'Lady', 'Ms', 'Countess'], 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    dataset['Title'] = dataset['Title'].replace(['Sir', 'Major'], 'Mr')
    dataset['Title'] = dataset['Title'].replace('^(?!.*(Miss|Mrs|Mr|Master)).*$', 'Rare',regex=True)
    
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.705882
2,Mr,0.159615
3,Mrs,0.793651
4,Rare,0.222222


In [64]:
title_mapping = {"Mr": 1, "Miss": 4, "Mrs": 2, "Master": 3, "Rare": 0}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,4
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,1


In [65]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

((891, 9), (418, 9))

In [66]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,22.0,1,0,7.2500,S,1
1,1,1,1,38.0,1,0,71.2833,C,2
2,1,3,1,26.0,0,0,7.9250,S,4
3,1,1,1,35.0,1,0,53.1000,S,2
4,0,3,0,35.0,0,0,8.0500,S,1


## Completing a numerical continuous feature¶


In [67]:
guess_ages = np.zeros((2,5))
guess_ages

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [68]:
for dataset in combine:
    for i in range(0,2):
        for j in range(0,5):
            guss_df = dataset[(dataset['Sex'] == i) & (dataset['Title'] == j +1)]['Age'].dropna()
            guss_age = guss_df.median()
            if guss_age != guss_age:
                guss_age = -5 #dataset[(dataset['Sex'] == i)]['Age'].dropna().median()
            guess_ages[i,j] = int(guss_age/0.5 + 0.5) * 0.5
    for i in range(0,2):
        for j in range(0,5):
            dataset.loc[(dataset.Age.isnull()) & (dataset.Sex == i ) & ( dataset.Title == j+1), 'Age'] = guess_ages[i,j]
    
    dataset.loc[(dataset.Age.isnull()), 'Age'] = dataset[(dataset['Sex'] == i)]['Age'].dropna().median()           
    dataset['Age'] = dataset['Age'].astype(int)
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,22,1,0,7.2500,S,1
1,1,1,1,38,1,0,71.2833,C,2
2,1,3,1,26,0,0,7.9250,S,4
3,1,1,1,35,1,0,53.1000,S,2
4,0,3,0,35,0,0,8.0500,S,1


In [69]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(-0.08, 16.0]",0.548077
1,"(16.0, 32.0]",0.327381
2,"(32.0, 48.0]",0.438424
3,"(48.0, 64.0]",0.434783
4,"(64.0, 80.0]",0.090909


In [70]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 4
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 1
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 0
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,AgeBand
0,0,3,0,3,1,0,7.2500,S,1,"(16.0, 32.0]"
1,1,1,1,2,1,0,71.2833,C,2,"(32.0, 48.0]"
2,1,3,1,3,0,0,7.9250,S,4,"(16.0, 32.0]"
3,1,1,1,2,1,0,53.1000,S,2,"(32.0, 48.0]"
4,0,3,0,2,0,0,8.0500,S,1,"(32.0, 48.0]"


In [71]:
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,3,1,0,7.2500,S,1
1,1,1,1,2,1,0,71.2833,C,2
2,1,3,1,3,0,0,7.9250,S,4
3,1,1,1,2,1,0,53.1000,S,2
4,0,3,0,2,0,0,8.0500,S,1


In [72]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch']

train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,FamilySize,Survived
3,3,0.724138
2,2,0.578431
1,1,0.552795
6,6,0.333333
0,0,0.303538
4,4,0.200000
5,5,0.136364
7,7,0.000000
8,10,0.000000


In [73]:
# for dataset in combine:
#     dataset['IsAlone'] = 0
#     dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

# train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

In [74]:
# for dataset in combine:
#     dataset['IsAlone'] = 0
#     dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

# train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()


In [75]:
train_df = train_df.drop(['Parch', 'SibSp'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp'], axis=1)
combine = [train_df, test_df]

train_df.head()

# train_df = train_df.drop([ 'FamilySize'], axis=1)
# test_df = test_df.drop([ 'FamilySize'], axis=1)
# combine = [train_df, test_df]

# train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize
0,0,3,0,3,7.2500,S,1,1
1,1,1,1,2,71.2833,C,2,1
2,1,3,1,3,7.9250,S,4,0
3,1,1,1,2,53.1000,S,2,1
4,0,3,0,2,8.0500,S,1,0


In [76]:
# for dataset in combine:
#     dataset['Age*Class'] = dataset.Age * dataset.Pclass

# train_df.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

In [77]:
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

'S'

In [78]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.339009


In [79]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 2, 'Q': 1} ).astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize
0,0,3,0,3,7.2500,0,1,1
1,1,1,1,2,71.2833,2,2,1
2,1,3,1,3,7.9250,0,4,0
3,1,1,1,2,53.1000,0,2,1
4,0,3,0,2,8.0500,0,1,0


In [80]:
for dataset in combine:
    dataset['Pclass'] = dataset['Pclass'].fillna(freq_port)
    
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [81]:
for dataset in combine:
    dataset['Pclass'] = dataset['Pclass'].map( {3: 0, 2: 1, 1: 2} ).astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize
0,0,0,0,3,7.2500,0,1,1
1,1,2,1,2,71.2833,2,2,1
2,1,0,1,3,7.9250,0,4,0
3,1,2,1,2,53.1000,0,2,1
4,0,0,0,2,8.0500,0,1,0


In [82]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize
0,892,0,0,2,7.8292,1,1,0
1,893,0,1,2,7.0000,0,2,1
2,894,1,0,1,9.6875,1,1,0
3,895,0,0,3,8.6625,0,1,0
4,896,0,1,3,12.2875,0,2,2


In [83]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(-0.001, 7.91]",0.197309
1,"(7.91, 14.454]",0.303571
2,"(14.454, 31.0]",0.454955
3,"(31.0, 512.329]",0.581081


In [84]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]
    
train_df.head(10)

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize
0,0,0,0,3,0,0,1,1
1,1,2,1,2,3,2,2,1
2,1,0,1,3,1,0,4,0
3,1,2,1,2,3,0,2,1
4,0,0,0,2,1,0,1,0
5,0,0,0,3,1,1,1,0
6,0,2,0,1,3,0,1,0
7,0,0,0,4,2,0,3,4
8,1,0,1,3,1,0,2,2
9,1,1,1,4,2,2,2,1


In [85]:
train_df.head(10)

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize
0,0,0,0,3,0,0,1,1
1,1,2,1,2,3,2,2,1
2,1,0,1,3,1,0,4,0
3,1,2,1,2,3,0,2,1
4,0,0,0,2,1,0,1,0
5,0,0,0,3,1,1,1,0
6,0,2,0,1,3,0,1,0
7,0,0,0,4,2,0,3,4
8,1,0,1,3,1,0,2,2
9,1,1,1,4,2,2,2,1


In [86]:
x = train_df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
train_df[:] = x_scaled

x = test_df.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
test_df[:] = x_scaled


combine = [train_df, test_df]
train_df.head(10)


,Survived,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize
0,0.0,0.0,0.0,0.75,0.000000,0.0,0.25,0.1
1,1.0,1.0,1.0,0.50,1.000000,1.0,0.50,0.1
2,1.0,0.0,1.0,0.75,0.333333,0.0,1.00,0.0
3,1.0,1.0,1.0,0.50,1.000000,0.0,0.50,0.1
4,0.0,0.0,0.0,0.50,0.333333,0.0,0.25,0.0
5,0.0,0.0,0.0,0.75,0.333333,0.5,0.25,0.0
6,0.0,1.0,0.0,0.25,1.000000,0.0,0.25,0.0
7,0.0,0.0,0.0,1.00,0.666667,0.0,0.75,0.4
8,1.0,0.0,1.0,0.75,0.333333,0.0,0.50,0.2
9,1.0,0.5,1.0,1.00,0.666667,1.0,0.50,0.1


In [87]:
train_df.head(10)

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize
0,0.0,0.0,0.0,0.75,0.000000,0.0,0.25,0.1
1,1.0,1.0,1.0,0.50,1.000000,1.0,0.50,0.1
2,1.0,0.0,1.0,0.75,0.333333,0.0,1.00,0.0
3,1.0,1.0,1.0,0.50,1.000000,0.0,0.50,0.1
4,0.0,0.0,0.0,0.50,0.333333,0.0,0.25,0.0
5,0.0,0.0,0.0,0.75,0.333333,0.5,0.25,0.0
6,0.0,1.0,0.0,0.25,1.000000,0.0,0.25,0.0
7,0.0,0.0,0.0,1.00,0.666667,0.0,0.75,0.4
8,1.0,0.0,1.0,0.75,0.333333,0.0,0.50,0.2
9,1.0,0.5,1.0,1.00,0.666667,1.0,0.50,0.1


## Model, predict and solve


In [88]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape


((891, 7), (891,), (418, 7))

In [89]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

78.34

In [90]:
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
1,Sex,2.029409
0,Pclass,1.744950
2,Age,1.220492
5,Title,1.168602
3,Fare,0.596717
4,Embarked,0.452102
6,FamilySize,-1.884110


In [91]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

83.16

In [92]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

85.86

In [93]:
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

79.91

In [94]:
perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

74.97

In [95]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

78.68

In [96]:
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

79.8

In [97]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

88.89

In [98]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

88.89

In [99]:
MLP = MLPClassifier(random_state=1, max_iter=300 )
MLP.fit(X_train, Y_train)
Y_pred = MLP.predict(X_test)
MLP.score(X_train, Y_train)
acc_MLP = round(MLP.score(X_train, Y_train) * 100, 2)
acc_MLP

83.95

In [100]:
    models = pd.DataFrame({'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree', 'MLP'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree, acc_MLP]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,88.89
8,Decision Tree,88.89
1,KNN,85.86
9,MLP,83.95
0,Support Vector Machines,83.16
4,Naive Bayes,79.91
6,Stochastic Gradient Decent,79.80
7,Linear SVC,78.68
2,Logistic Regression,78.34
5,Perceptron,74.97


In [101]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('../out/submission.csv', index=False)

In [102]:
accuracy_score(result_df['Survived'], Y_pred, normalize=False)

369

In [145]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses


In [169]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2)
])

In [170]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [172]:
model.fit(X_train.values, Y_train.values, epochs=20)

Train on 891 samples
Epoch 1/20
891/891 [==============================] - 0s 105us/sample - loss: 0.3992 - accuracy: 0.8159
Epoch 2/20
891/891 [==============================] - 0s 115us/sample - loss: 0.3967 - accuracy: 0.8249
Epoch 3/20
891/891 [==============================] - 0s 141us/sample - loss: 0.3914 - accuracy: 0.8361
Epoch 4/20
891/891 [==============================] - 0s 137us/sample - loss: 0.3911 - accuracy: 0.8339
Epoch 5/20
891/891 [==============================] - 0s 134us/sample - loss: 0.3854 - accuracy: 0.8316
Epoch 6/20
891/891 [==============================] - 0s 114us/sample - loss: 0.3824 - accuracy: 0.8384
Epoch 7/20
891/891 [==============================] - 0s 95us/sample - loss: 0.3881 - accuracy: 0.8272
Epoch 8/20
891/891 [==============================] - 0s 104us/sample - loss: 0.3996 - accuracy: 0.8283
Epoch 9/20
891/891 [==============================] - 0s 143us/sample - loss: 0.4053 - accuracy: 0.8373
Epoch 10/20
891/891 [=======================

In [167]:
Y_pred = model.predict(X_test)


In [168]:
Y_pred

array([[ 7.72662759e-01, -9.49271619e-01],
       [-4.77854192e-01,  2.14862168e-01],
       [ 5.67640245e-01, -5.96419334e-01],
       [ 4.37227070e-01, -8.76711011e-01],
       [-2.29029760e-01, -6.56734481e-02],
       [ 2.93982655e-01, -8.44648898e-01],
       [-8.69355977e-01,  8.19222867e-01],
       [ 4.17080283e-01, -8.22601259e-01],
       [-1.22475088e+00,  1.05164623e+00],
       [ 8.37938607e-01, -1.18060923e+00],
       [ 7.57746041e-01, -1.21387517e+00],
       [-7.93308914e-02, -2.17302173e-01],
       [-2.13883972e+00,  1.96316719e+00],
       [ 6.97875202e-01, -9.55747664e-01],
       [-2.11587405e+00,  1.96584749e+00],
       [-1.86023200e+00,  1.63818562e+00],
       [ 3.75999659e-01, -5.05438507e-01],
       [ 5.32798171e-01, -7.32834816e-01],
       [-1.44211069e-01, -6.38893619e-03],
       [-1.04100645e+00,  9.20889616e-01],
       [-3.24509256e-02, -7.03227371e-02],
       [-6.67738974e-01,  1.53585106e-01],
       [-2.14980960e+00,  2.00211954e+00],
       [-3.